# Most basic time decomposition simulation
We assume a flare with fixed physical parameters and two basis light curves.

The basis light curves are fractional browninan motion with $H = 0.5$ and $H = 0.955$.

The instrument response matrix is ignored for now.

The background is randomly distributed around a mean value,
with $\sigma = 4\mu$,
such that the nonthermal component appears washed out above 50 keV.

The physical parameters are as follows

**Thermal, isothermal**
- $\text{T} = 20$ MK
- $\text{EM} = 10^{49} \text{ cm}^{-3}$

**Nonthermal, thick target**
- $\varphi_e = 2\times 10^{35}$ electron/second
- $\gamma = 4$
- $\text{E}_c = 16$ keV

In [15]:
import os
import copy

from astropy import units as u
from astropy import visualization as viz
from matplotlib import pyplot as plt
import numpy as np

import scipy.stats as st
from yaff import common_models as cm
from yaff.fitting import Parameter
from yaff import plotting

from tedec import fractional_brownian_motion as fbm
from tedec import decomp

%matplotlib qt
plt.style.use('/home/william/dev/nice.mplstyle')

## Generate some basis light curves to use later

In [16]:
seed = 132457
np.random.seed(seed)
time_bin = 0.1
integration = 30
steps = int(integration / time_bin)
thermal_basis = fbm.make_timeseries(num=steps, hurst=0.955)
nonthermal_basis = fbm.make_timeseries(num=steps, hurst=0.5)

/home/william/dev/yaff/tedec/tedec/fractional_brownian_motion.py:15: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  sigma = scil.sqrtm(gamma)


In [17]:
def rebin_clumps(histogram, clump_size):
    ret = np.zeros(histogram.size // clump_size)
    for i in range(0, histogram.size, clump_size):
        ret[i // clump_size] = histogram[i:i+clump_size].sum()
    return ret

# Bin down the time granularity
real_dt = 0.5
bin_down_factor = int(real_dt / time_bin)
thermal_basis = rebin_clumps(thermal_basis, bin_down_factor)
nonthermal_basis = rebin_clumps(nonthermal_basis, bin_down_factor)

In [18]:
fig, ax = plt.subplots()
axx = ax.twinx()
t = np.arange(thermal_basis.size + 1)
ax.stairs(thermal_basis, t, color='red', label='thermal')
ax.legend(loc='lower right')
ax.set(ylabel='thermal magnitude')

axx.stairs(nonthermal_basis, t, color='black', label='nonthermal')
axx.legend(loc='upper left')
axx.set(ylabel='nonthermal magnitude')

plt.show()

In [19]:
def normalize(s):
    s -= s.min()
    s += 4
    return np.nan_to_num(s / s.sum())

norm_th = normalize(thermal_basis)
norm_nth = normalize(nonthermal_basis)

fig, ax = plt.subplots()
ax.stairs(norm_th, t, label='thermal')
ax.stairs(norm_nth, t, label='nonthermal')
plt.show()

## Take the basis lightcurves and use them to scale $\text{EM}$ and $\varphi_e$

In [21]:
# Vary EM by factor of 3
th_scale = norm_th - norm_th.min()
th_scale /= th_scale.max()
th_scale *= 2
th_scale += 1

# Vary electron flux by max factor of 4
nth_scale = norm_nth - norm_nth.min()
nth_scale /= nth_scale.max()
nth_scale *= 3
nth_scale += 1

In [76]:
nth_scale

array([2.97196729, 3.06751079, 3.40942778, 3.28546286, 3.25268931,
       3.43802609, 3.43394631, 3.38279776, 2.92631971, 3.07095971,
       3.08672616, 3.42446196, 3.57238002, 3.28488924, 2.80847212,
       2.56244872, 2.37049819, 2.18380881, 2.53313353, 2.85921664,
       3.11741048, 3.15416922, 3.17095508, 3.28971873, 2.89072315,
       2.37306214, 2.33974315, 2.28800352, 1.9842158 , 1.94219134,
       1.83357842, 1.50571264, 1.38718752, 1.30973273, 1.71854891,
       1.55036144, 1.19862248, 1.        , 1.1063555 , 1.29151909,
       1.74089145, 2.34161816, 3.01100736, 3.25056897, 3.04318813,
       3.35921744, 3.33289609, 3.24339505, 3.32886687, 3.35711604,
       3.44128407, 3.46149403, 3.5974426 , 3.50274802, 3.69180017,
       3.47941335, 3.43705734, 3.72017958, 4.        , 3.65323517])

In [39]:
thermal_physical_params = {
    'temperature': Parameter(20 << u.MK, True),
    'emission_measure': Parameter(1 << (1e49 * u.cm**-3), True)
}

nonthermal_physical_params = {
    'electron_flux': Parameter(6 << (1e35 * u.electron / u.s), True),
    'spectral_index': Parameter(4 << u.one, True),
    'cutoff_energy': Parameter(25 << u.keV, True)
}

energies = np.geomspace(3, 100, num=40) << u.keV

all_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': (thermal_physical_params | nonthermal_physical_params)
}

thermal_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': thermal_physical_params
}

nonthermal_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': nonthermal_physical_params
}

In [40]:
def model(params: cm.ArgsT):
    return cm.thermal(params) + cm.thick_target(params)

In [60]:
de = energies[1:] - energies[:-1]
area = 10 << u.cm**2

def flux_to_photons(func, args):
    flux_unit = (u.ph / u.cm**2 / u.s / u.keV)
    return ((func(args) << flux_unit) * de * (real_dt << u.s) * area).to(u.ph)


thermal_truth = list()
nonthermal_truth = list()
spectrogram = list()
for i in range(th_scale.size):
    # Update the EM and phi_e by the scale factor
    args = copy.deepcopy(all_args)
    args['parameters']['emission_measure'].value *= th_scale[i]
    args['parameters']['electron_flux'].value *= nth_scale[i]

    thermal_truth.append(flux_to_photons(cm.thermal, args))
    nonthermal_truth.append(flux_to_photons(cm.thick_target, args))
    spectrogram.append(flux_to_photons(model, args))

def strip_unit(a):
    return (a << u.ph).value.T

spectrogram = strip_unit(spectrogram)
thermal_truth = strip_unit(thermal_truth)
nonthermal_truth = strip_unit(nonthermal_truth)

In [62]:
tests = np.arange(spectrogram.shape[0])

fig, ax = plt.subplots()
for test in tests:
    ax.stairs(spectrogram[test], t)
ax.set(xlabel='time (s)', ylabel='photons incident')
plt.show()

In [63]:
energy_mids = (energies[:-1] + np.diff(energies)/2).to_value(u.keV)
closest = lambda a, v: np.argmin(np.abs(a - v))

'''
Let's say we have a 5 uCurie Ba133 source on board.
For X-rays that's about 2e5 count/second.
There will be lines at 4 keV, 31 keV, and 81 keV
'''
count_rate = 100
num_time_bins = thermal_basis.size
noise = (count_rate * num_time_bins * real_dt) * np.ones_like(total_photons)
noise[closest(energy_mids, 4):closest(energy_mids, 6)] *= 100
noise[closest(energy_mids, 29):closest(energy_mids, 32)] *= 10
noise[closest(energy_mids, 80):closest(energy_mids, 84)] *= 5

background = st.norm.rvs(loc=noise, scale=np.sqrt(noise), size=(thermal_basis.size, noise.size)).T / num_time_bins

In [64]:
from matplotlib import colors as mcol

fig, ax = plt.subplots()
# ax.pcolormesh(t, energies, background)
norm = mcol.LogNorm(vmin=background.min(), vmax=background.max())
ax.pcolormesh(t, energies, background, norm=norm)
ax.set(xlabel='time', ylabel='energy')
plt.show()

In [66]:
# Add counting statistics & systematics onto the photon data
systematic = 0.05
data = st.norm.rvs(loc=spectrogram, scale=np.sqrt(spectrogram + (systematic * spectrogram)**2))

# Insert the background from the radioactive source
noisy_data = data + background
if noisy_data.min() < 0:
    raise ValueError("Can't have negative counts")

In [78]:
from matplotlib import colors as mcol

fig, ax = plt.subplots()
norm = mcol.LogNorm(vmin=100, vmax=noisy_data.max())
pcm = ax.pcolormesh(t, energies, noisy_data, norm=norm)
fig.colorbar(pcm, label='photons')
ax.set(xlabel='time', ylabel='energy')
plt.show()

In [68]:
reconstructed = spectrogram.sum(axis=1)
fig, ax = plt.subplots()

with viz.quantity_support():
    ax.stairs(reconstructed, energies, label='reconstructed data', linestyle='dashed', color='red')
    ax.stairs(noisy_data.sum(axis=1), energies, label='noisy, reconstructed data', linestyle='dashed', color='orange')
    ax.stairs(background.sum(axis=1), energies, label='background')

ax.legend()
ax.set(xscale='log', yscale='log')
plt.show()

In [73]:
nearest = lambda a, v: np.argmin(np.abs(a - v))

thermal_index = nearest(energy_mids, 4)
nonthermal_index = nearest(energy_mids, 19)

pack = decomp.DataPacket(
    data=noisy_data,
    basis_timeseries=[
        noisy_data[thermal_index],
        noisy_data[nonthermal_index],
    ],
    constant_offset=True
)

systematic = 0.1
ret = decomp.bootstrap(
    pack,
    errors=np.sqrt(noisy_data + (systematic * noisy_data)**2),
    num_iter=1000
)

In [82]:
th_mean = ret[:, 0, :].mean(axis=0) << u.ph
th_std = ret[:, 0, :].std(axis=0) << u.ph
nth_mean = ret[:, 1, :].mean(axis=0) << u.ph
nth_std = ret[:, 1, :].std(axis=0) << u.ph

# scale by # time bins (need to update)
bkg_part = ret[:, 2, :] * th_scale.size
bkg_mean = bkg_part.mean(axis=0) << u.ph
bkg_std = bkg_part.std(axis=0) << u.ph

fig, ax = plt.subplots()

with viz.quantity_support():
    ax.stairs(thermal_truth.sum(axis=1), energies, label='true thermal spectrum')
    ax.stairs(nonthermal_truth.sum(axis=1), energies, label='true nonthermal spectrum')
    ax.stairs(background.sum(axis=1), energies, label='true background')
    
    num_sigma = 2
    plotting.stairs_with_error(energies, th_mean, num_sigma*th_std, ax=ax, label='decomposed thermal')
    plotting.stairs_with_error(energies, nth_mean, num_sigma*nth_std, ax=ax, label='decomposed nonthermal')
    plotting.stairs_with_error(energies, bkg_mean, num_sigma*bkg_std, ax=ax, label='bkg decom')

ax.legend()
ax.set(xscale='log', yscale='log', ylim=(1e3, None))
plt.show()